In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import pandas as pd
from tqdm import tqdm

In [ ]:
model_id = "scb10x/llama-3-typhoon-v1.5x-70b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

2025-06-23 09:02:21.058640: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750669341.250202      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750669341.305977      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


...

model-00020-of-00030.safetensors:   0%|          | 0.00/4.66G [00:00<?, ?B/s]

model-00021-of-00030.safetensors:   0%|          | 0.00/4.66G [00:00<?, ?B/s]

model-00022-of-00030.safetensors:   0%|          | 0.00/4.66G [00:00<?, ?B/s]

model-00023-of-00030.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00024-of-00030.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

In [ ]:
def generate_answers(csv_path, tokenizer, model):
    df = pd.read_csv(csv_path)

    answers = []
    for query in tqdm(df['query'], desc="Generating answers"):
        messages = [
            {"role": "system", "content": "You are a AI assistant that can only answer A, B, C, D, FALL or RISE"},
            {"role": "user", "content": query},
        ]

        input_ids = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="pt",
            enable_thinking=False  # Optional depending on tokenizer
        ).to(model.device)

        outputs = model.generate(
            input_ids,
            max_new_tokens=1,
            do_sample=True,
            temperature=0.6,
            top_p=0.95,
        )

        response = outputs[0][input_ids.shape[-1]:]
        answer = tokenizer.decode(response, skip_special_tokens=True).strip()
        answers.append(answer)

    df['answer'] = answers
    return df

In [ ]:
df_with_answers = generate_answers("/kaggle/input/financial-analysis-agent/test.csv", tokenizer, model)

In [ ]:
df_with_answers = df_with_answers.drop('query', axis=1)

In [ ]:
df_with_answers.to_csv("output.csv", index=False)